In [1]:
from zmeasure import main
from zmeasure.instruments import FakeInstrument
from zmeasure import utility
import time
# import all necessary packages and drivers
import numpy as np
import pandas as pd
import re

from functools import  partialmethod,partial
from zmeasure.main import (start_process, stop_process,pause_process,resume_process, 
                                  switch_file, clear_plot,update_PID)
# import matplotlib
from zmeasure.utility import (cls_all, error_response,get_d,get_disp,get_current_disp,
                     get_current_strain,wait_until,get_highest_indx_file, reset_all,get_cernox_temp,mkdir)
# global data_to_save
from zmeasure.read_write_tab import read_file,read_idx,read_max_idx
# %matplotlib inline
from zmeasure.experiments import sweep
from zmeasure.experiments.sweep import DoubleSweeper


In [2]:

from multiprocessing import Manager
manager = Manager()
shared_value1 = manager.Value('d', 0.0)  # shared float
shared_value2 = manager.Value('d', 0.0)  # shared float

fake_instru1 = FakeInstrument(name='Temp',shared=shared_value1)
fake_instru2 = FakeInstrument(name='Field',shared=shared_value2)

if_plot = False
if True: #if_plot use
    #plot configuration
    ncol, nrow = 3,3
    #all plot x,y s
    plot_axes_col_names = [
        ['sys:time','Temp'],
        ['sys:time','Field'],
    ]

#data saving
data_folder = mkdir('data')
default_file = 'test_data_file' # first file to save

#all data entries saved in files. format as "instrument_name:reading_name" reading name can be reset in drivers
read_col_names = [
    # sys:time,
    'Temp',
    'Field',
]

#reading functions are defined in drivers
read_funcs = [
    fake_instru1.read,
    fake_instru2.read,
]

# test your reading functions, no data selection
def read_all(funcs_to_call,read_col_names=read_col_names):
        all_data = []
        all_data_name = []
        for i in range(len(funcs_to_call)):
            t1 = time.time()
            data,name = error_response(funcs_to_call[i])
            print(i,time.time()-t1)
            print(i,name,data,{key:value for key,value in zip(name,data)})
            all_data += data
            all_data_name += name
        return np.array(all_data), np.array(all_data_name)
t = time.time()
all_data, all_data_name = read_all(read_funcs)
print('overall takes',time.time()-t)
if any([not (x in all_data_name) for x in read_col_names]):
    print('columns not in reading')
else:
    print('all columns read')

# if any([not (x in read_col_names) for x in np.array(plot_axes_col_names).flatten()]):
#     print('columns not in reading')
# else:
#     print('all plot axis recorded')
print({key:value for key,value in zip(all_data_name,all_data,)})
# reset_all(allInstrumentList)
# for it in gpibInstrumentList:
#     print(it._resource_manager)

('data',)
data already exists
data already exists, description file already exists
0 0.10384893417358398
0 ['Temp'] [0.0] {'Temp': 0.0}
1 0.10525298118591309
1 ['Field'] [0.0] {'Field': 0.0}
overall takes 0.20936870574951172
all columns read
{'Temp': 0.0, 'Field': 0.0}


In [3]:
clear_funcs = []
allInstrumentList = []

In [4]:
cls_all(clear_funcs)
reset_all(allInstrumentList)
t0 = time.time()#wait until test
t1 = t0
start_process(  ncol=ncol,
                nrow=nrow,
                data_folder=data_folder,
                default_file='test_before_cd',
                read_col_names=read_col_names,
                plot_axes_col_names=plot_axes_col_names,
                read_funcs=read_funcs,
                if_plot=if_plot,
                # PID_control_map = PID_control_map,
                # init_PID_value = init_PID_value,
                clear_funcs=clear_funcs,
                max_plot_N=100,
                start_time=t0
             )

In [6]:
class Sweeper(sweep.Sweeper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pause_func = pause_process
        self.resume_func = resume_process
        self.stop_func = stop_process
        self.switch_file_func = switch_file
        self.read_csv_func = partial(read_max_idx,folder=data_folder)
        
class field_sweep(Sweeper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label = "field"
        self.sweep_func = fake_instru2.write
        self.get_current_func = lambda: fake_instru2.read()[0][0]
        self.delay_resume = 1
        self.wait_first = 0
        self.wait_rest = 1
        self.safe_rate = 100
        

class temp_sweep(Sweeper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.label = "temp"
        self.sweep_func = fake_instru1.write
        self.get_current_func = lambda: fake_instru1.read()[0][0]
        self.delay_resume = 1
        self.wait_first = 0
        self.wait_rest = 1
        self.safe_rate = 1
        
temp_sweeper = temp_sweep()
field_sweeper = field_sweep()

double_sweeper = DoubleSweeper(temp_sweeper, field_sweeper)
double_sweeper.run_double_sweep(
    outer_sweep_values=[0, 10, 20],
    outer_sweep_rates=[1, 2, 3],
    outer_entry_names=['Temp'],   
    outer_entry_tolerances=[0.1],
    
    inner_sweep_values=[0, 1, None,2],
    inner_sweep_rates=[1, 2, None, 3],
    inner_entry_names=['Field'],
    inner_entry_tolerances=[0.1],

    sweep_back = True
)

0
switch file
switch file done
0
switch file
switch file done
0
1
1
switch file
switch file done
2
2
switch file
switch file done
1
0
switch file
switch file done
0
switch file
switch file done
0
1
1
switch file
switch file done
2
2
switch file
switch file done


KeyboardInterrupt: 

In [7]:
stop_process()

In [7]:
type(None)

NoneType

In [9]:
temp_sweeper.switch_file_func('test')